In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from scipy.sparse import *
from scipy.sparse.linalg import svds

# 곡-플레이리스트 행렬에서 계산된 곡 간의 유사도 테스트

In [87]:
#플레이 리스트
train = pd.read_json('../data/train.json')

#곡 정보
song_meta = pd.read_json('../data/song_meta.json')

In [88]:
train.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000


### 곡 id를 다시 인덱싱

In [89]:
# 플레이리스트의 모든 곡을 하나로 합치고 set으로 중복을 제거한다.
all_songs = set(np.concatenate(train['songs']))

#중복을 제외한 플레이리스트 내 등장한 전체 곡 수
n_songs = len(all_songs)

#새 인덱스와 기존 인덱스간 인코딩 디코딩을 위한 딕셔너리 초기화.
# song_to_newid[원본인덱스] -> 바뀐 인덱스
# newid_to_song[바뀐인덱스] -> 원본인덱스
song_to_newid  = dict()
newid_to_song = dict()

In [90]:
# 위 딕셔너리를 채운다.
for i, t in enumerate(all_songs):
    song_to_newid[t] = i
    newid_to_song[i] = t

In [91]:
#기존 곡 id를 새 id로 바꾼 컬럼을 만든다.
train['song_newid'] = train['songs'].apply(lambda x: [song_to_newid[song] for song in x])
display(train['song_newid'])

0         [456704, 112732, 333158, 488440, 258853, 12127...
1         [375894, 587314, 431997, 104605, 338568, 21226...
2         [72132, 240434, 144495, 161861, 307991, 222934...
3         [342495, 169897, 469393, 250235, 383169, 87161...
4         [138494, 481095, 4399, 560460, 255877, 87448, ...
                                ...                        
115066    [373452, 383795, 531854, 448795, 601027, 33646...
115067    [279203, 187803, 464493, 208798, 287747, 20234...
115068    [43717, 216378, 217755, 322592, 199796, 603802...
115069    [463690, 528384, 298561, 362605, 529162, 26262...
115070    [22532, 396641, 281634, 78058, 117563, 124743,...
Name: song_newid, Length: 115071, dtype: object

### playlist - song 테이블 구성

`csr_matrix`를 구성하기위해 아래 3개가 필요하다.  
`csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])`
- row : 각 플레이리스트에 있는 곡 수만큼 반복되는 플레이리스트 인덱스들
- col : 각 플레이리스트에 있는 곡들의 id 리스트
- data : 각 플레이리스트 곡 값 (우리는 평점이 아니므로 전부 1)

### row

In [92]:
# 각 플레이리스트 별 곡 수가 필요함
train['song_cnt'] = train['songs'].map(lambda x : len(x))

#각 플레이리스트 id를 각 플레이리스트 곡 수 만큼 반복
row = np.repeat(range(len(train)), train['song_cnt'])
row.shape

(5285871,)

### col

In [93]:
#각 플레이 리스트에 있는 곡들을 전부 붙이면 된다.
col =  np.concatenate(train['song_newid'])
col.shape

(5285871,)

### data

In [94]:
# 우리는 평점에 대한 정보가 없으므로 data는 전부 1인 array가 된다.
data = np.ones(col.shape[0])

## sparse matrix

In [95]:
# 위 데이터로 희소행렬을 만든다.
ply_song_table = csr_matrix((data, (row,col)), shape= (len(train), n_songs))

In [96]:
#행이 플레이리스트, 열이 곡인 행렬
ply_song_table

<115071x615142 sparse matrix of type '<class 'numpy.float64'>'
	with 5285871 stored elements in Compressed Sparse Row format>

# 곡 간의 유사도 계산

In [97]:
#곡 간 유사도 계산을 위해 곡이 행으로 플레이리스트가 열로 와야한다.
song_ply_table = ply_song_table.transpose()

#코사인 유사도 계산.
song_sim = cosine_similarity(song_ply_table, song_ply_table, dense_output = False)
#곡 수 x 곡 수 크기의 유사도 행렬이 만들어짐.

아이유 밤편지와 유사한 곡 30곡을 뽑아보자.  
밤편지의 원본 id는 144663이다

In [98]:
# 밤편지의 원본 id를 song_to_newid로 새로 부여된 인덱스로 바꿈
# song_sim[바뀐인덱스] - 밤편지와 다른 곡들간의 유사도를 toarray()로 spaese 형식에서 ndarray로 변환
# reshape(-1)로 행렬을 벡터로 변환.
# argsort()[::-1][:30]로 상위 30개의 곡 인덱스 추출.
top30 =  song_sim[song_to_newid[144663]].toarray().reshape(-1).argsort()[::-1][:30]

In [99]:
# 추출된 곡 인덱스를 원본 인덱스로 바꾼다.
top30_song = [newid_to_song[x] for x in top30]
top30_song

[144663,
 140920,
 174749,
 305045,
 185174,
 648635,
 532347,
 541682,
 367963,
 346058,
 612516,
 459165,
 224921,
 357367,
 301392,
 423594,
 19533,
 70185,
 468613,
 478754,
 15341,
 8719,
 601037,
 334539,
 627363,
 205247,
 123541,
 501822,
 244565,
 246531]

In [100]:
song_meta.iloc[top30_song][['song_name','artist_name_basket']]

,song_name,artist_name_basket
144663,밤편지,[아이유]
140920,팔레트 (Feat. G-DRAGON),[아이유]
174749,비도 오고 그래서 (Feat. 신용재),[헤이즈 (Heize)]
305045,가을 아침,[아이유]
185174,사랑이 잘 (With 오혁),[아이유]
648635,이름에게,[아이유]
532347,이런 엔딩,[아이유]
541682,솔직하게 말해서 나,[김나영]
367963,좋니,[윤종신]
346058,가을 타나 봐,[바이브]


## ALS 추천 방식

implicit 라이브러리 사용  
https://implicit.readthedocs.io/en/latest/als.html

In [72]:
from implicit.als import AlternatingLeastSquares as ALS

In [81]:
als_model = ALS(factors=30, regularization=0.08)
als_model.fit(ply_song_table.T * 15.0)

  0%|          | 0/15 [00:00<?, ?it/s]

In [82]:
model = ALS(use_gpu=False)
model.user_factors = als_model.user_factors
model.user_factors.shape

(115071, 30)

In [83]:
model.item_factors = als_model.item_factors
model.item_factors.shape

(615142, 30)

In [84]:
topsong = model.similar_items(song_to_newid[144663],N=30)

In [85]:
topsong_30 = [newid_to_song[idx] for idx, _ in topsong]

In [86]:
song_meta.iloc[topsong_30][['song_name','artist_name_basket']]

,song_name,artist_name_basket
144663,밤편지,[아이유]
305045,가을 아침,[아이유]
357367,비,[폴킴]
174749,비도 오고 그래서 (Feat. 신용재),[헤이즈 (Heize)]
351342,끝,[권진아]
532347,이런 엔딩,[아이유]
185174,사랑이 잘 (With 오혁),[아이유]
611737,11:11,[태연 (TAEYEON)]
471691,오랜 날 오랜 밤,[AKMU (악동뮤지션)]
70185,잠 못 드는 밤 비는 내리고,[아이유]
